In [1]:
import h2o
from h2o.automl import H2OAutoML

In [3]:
# Start the H2O cluster (locally)
#h2o.init()

# leave 1/3 for the RAM to XGBoost
h2o.init(max_mem_size = "10G")


#train = h2o.import_file(path="/Users/sarayabesi/Desktop/final_fixations_train.csv")
#test = h2o.import_file(path="/Users/sarayabesi/Desktop/final_fixations_test.csv")
data = h2o.import_file(path="/Users/sarayabesi/Documents/research-poly/paper_2/model_3/data_model_3.csv")

# Split the data into Train/Test/Validation with Train having 70% and test and validation 15% each
#train,test,valid = data.split_frame(ratios=[.7, .15])
train,test = data.split_frame(ratios=[.8])




Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,7 days 1 hour 0 mins
H2O_cluster_timezone:,America/Toronto
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,8 months and 14 days
H2O_cluster_name:,H2O_from_python_sarayabesi_dbu1zu
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8.59 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [4]:
'number of rows in the whole dataset:' , len(data)

('number of rows in the whole dataset:', 222)

In [5]:
'number of rows in train:' , len(train)

('number of rows in train:', 181)

In [8]:
'number of rows in test:' , len(test)

('number of rows in test:', 53)

In [9]:
data.head(5)

﻿participant_id,task_number,Task_difficulty,Priority_Low_High,Expertise_Expert_Novice,is_accepted
101,T1,7,high,senior,1
101,T2,7,low,senior,0
101,T3,7,low,junior,1
101,T4,7,high,junior,0
101,T5,7,low,senior,1


In [10]:
train.head()

﻿participant_id,task_number,Task_difficulty,Priority_Low_High,Expertise_Expert_Novice,is_accepted
101,T2,7,low,senior,0
101,T3,7,low,junior,1
101,T5,7,low,senior,1
102,T1,6,high,junior,1
102,T2,6,high,junior,1
102,T3,6,low,junior,1
102,T5,6,high,senior,0
102,T6,6,low,senior,1
103,T2,2,low,junior,0
103,T3,2,high,senior,0


In [12]:
# Identify predictors and response
x = data.columns[2:] # exclude 'participant_id' , 'Task'
y = "is_accepted"
x.remove(y)

In [14]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [15]:
# Run AutoML for 20 base models
aml = H2OAutoML(max_models=4, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |
15:23:46.810: AutoML: XGBoost is not available; skipping it.


15:23:47.417: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 169.0.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: DRF_1_AutoML_1_20231002_152346


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    33                 33                          4185                   1            6            3.39394       2             10            5.45455

ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.25007769715345046
RMSE: 0.5000776911175407
LogLoss: 0.693355694010239
Mean Per-Class Error: 0.48148148148148145
AUC: 0.5402637485970819
AUCPR: 0.542864035812796
Gini: 0.08052749719416386

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.38962576939509463
       0    1    Error    Rate
-----  ---  ---  -------  ------------
0      3    78   0.963    (78.0/81.0)
1      0    88   0        (0.0/88.0)
Total  3    166  0.4615   (78.0/169.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.389626     0.692913  165
max f2                       0.389626     0.849421  165
max f0point5                 0.389626     0.585106  165
max accuracy                 0.533611     0.568047  66
max precision                0.652083     1         0
max recall                   0.389626     1         165
max specificity              0.652083     1         0
max absolute_mcc             0.5538       0.149207  44
max min_per_class_accuracy   0.522543     0.54321   84
max mean_per_class_accuracy  0.533611     0.572461  66
max tns                      0.652083     81        0
max fns                      0.652083     87        0
max fps                      0.327097     81        168
max tps                      0.389626     88        165
max tnr                      0.652083     1         0
max fnr                      0.652083     0.988636  0
max fpr                      0.327097     1         168
max tpr                      0.389626     1         165

Gains/Lift Table: Avg response rate: 52.07 %, avg score: 51.95 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0118343                   0.627917           0.960227  0.960227           0.5              0.64722   0.5                         0.64722             0.0113636       0.0113636                  -3.97727  -3.97727           -0.000982043
2        0.0236686                   0.619873           0         0.480114           0                0.620606  0.25                        0.633913            0               0.0113636                  -100      -51.9886           -0.0256734
3        0.035503                    0.614769           1.92045   0.960227           1                0.617157  0.5                         0.628328            0.0227273       0.0340909                  92.0455   -3.97727           -0.00294613
4        0.0414201                   0.611435           0         0.823052           0                0.613394  0.428571                    0.626194            0               0.0340909                  -100      -17.6948           -0.0152918
5        0.0532544                   0.609884           0         0.640152           0                0.610564  0.333333                    0.622721            0            

In [16]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows 

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
DRF_1_AutoML_1_20231002_152346,0.568533,0.685845,0.555968,0.46149,0.496334,0.246348
GBM_3_AutoML_1_20231002_152346,0.52897,0.697401,0.529825,0.5,0.501941,0.251945
GBM_2_AutoML_1_20231002_152346,0.522236,0.705129,0.514397,0.5,0.505404,0.255433
StackedEnsemble_BestOfFamily_1_AutoML_1_20231002_152346,0.444795,0.69522,0.474497,0.487163,0.501032,0.251033
StackedEnsemble_AllModels_1_AutoML_1_20231002_152346,0.383347,0.706013,0.447171,0.5,0.506377,0.256418
GLM_1_AutoML_1_20231002_152346,0.363286,0.707176,0.447072,0.5,0.506942,0.25699


In [17]:
# prediction on the leader model
preds = aml.leader.predict(test)

drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [18]:
preds

predict,p0,p1
1,0.539208,0.460792
1,0.425305,0.574695
1,0.425305,0.574695
1,0.421288,0.578712
0,0.708096,0.291904
1,0.46104,0.53896
1,0.49935,0.50065
1,0.505776,0.494224
1,0.474891,0.525109
1,0.475919,0.524081


In [19]:
perf = aml.leader.model_performance(test)
perf

ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.25590005065492744
RMSE: 0.5058656448652423
LogLoss: 0.7046583940928707
Mean Per-Class Error: 0.4782608695652174
AUC: 0.3536231884057971
AUCPR: 0.4944126458710507
Gini: -0.2927536231884058

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4567087373950265
       0    1    Error    Rate
-----  ---  ---  -------  -----------
0      1    22   0.9565   (22.0/23.0)
1      0    30   0        (0.0/30.0)
Total  1    52   0.4151   (22.0/53.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.456709     0.731707  20
max f2                       0.456709     0.872093  20
max f0point5                 0.456709     0.630252  20
max accuracy                 0.456709     0.584906  20
max precision                0.456709     0.576923  20
max recall                   0.456709     1         20
max specificity              0.593766     0.913043  0
max absolute_mcc             0.50065      0.337009  13
max min_per_class_accuracy   0.524081     0.391304  10
max mean_per_class_accuracy  0.456709     0.521739  20
max tns                      0.593766     21        0
max fns                      0.593766     28        0
max fps                      0.291904     23        21
max tps                      0.456709     30        20
max tnr                      0.593766     0.913043  0
max fnr                      0.593766     0.933333  0
max fpr                      0.291904     1         21
max tpr                      0.456709     1         20

Gains/Lift Table: Avg response rate: 56.60 %, avg score: 51.85 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0754717                   0.593766           0.883333  0.883333           0.5              0.593766  0.5                         0.593766            0.0666667       0.0666667                  -11.6667  -11.6667           -0.0202899
2        0.113208                    0.583508           0.883333  0.883333           0.5              0.584707  0.5                         0.590746            0.0333333       0.1                        -11.6667  -11.6667           -0.0304348
3        0.169811                    0.578712           1.17778   0.981481           0.666667         0.578712  0.555556                    0.586735            0.0666667       0.166667                   17.7778   -1.85185           -0.00724638
4        0.207547                    0.567205           0.883333  0.963636           0.5              0.574695  0.545455                    0.584546            0.0333333       0.2                        -11.6667  -3.63636           -0.0173913
5        0.339623                    0.53896            0.504762  0.785185           0.285714         0.543886  0.444444                    0.568734            0.0666667       0.266667                   -49.5238  -21.4815           -0.168116
6        0.396226                    0.525884           0         0.673016           0                0.528987  0.380952                    0.563055            0               0.266667                   -100      -32.6984           -0.298551
7        0.584906                    0.517653           1.06      0.797849           0.6              0.52097   0.451613                    0.54948             0.2             0.466667                   6         -20.2151           -0.272464
8        0.622642                    0.512216           0.883

In [20]:
# Get leaderboard with all possible columns
# log loss calculation is for validation data
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
lb

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
DRF_1_AutoML_1_20231002_152346,0.568533,0.685845,0.555968,0.46149,0.496334,0.246348,76,0.031137,DRF
GBM_3_AutoML_1_20231002_152346,0.52897,0.697401,0.529825,0.5,0.501941,0.251945,33,0.019688,GBM
GBM_2_AutoML_1_20231002_152346,0.522236,0.705129,0.514397,0.5,0.505404,0.255433,37,0.021466,GBM
StackedEnsemble_BestOfFamily_1_AutoML_1_20231002_152346,0.444795,0.69522,0.474497,0.487163,0.501032,0.251033,327,0.031141,StackedEnsemble
StackedEnsemble_AllModels_1_AutoML_1_20231002_152346,0.383347,0.706013,0.447171,0.5,0.506377,0.256418,215,0.023306,StackedEnsemble
GLM_1_AutoML_1_20231002_152346,0.363286,0.707176,0.447072,0.5,0.506942,0.25699,10,0.0142,GLM
